In [ ]:
import numpy as np
import math
import torch
from PIL import Image
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
classes = 100
train_total = 50000

#modified from: 
#https://github.com/kekmodel/FixMatch-pytorch/blob/master/dataset/cifar.py 
def x_u_split(labels, labeled_per_class):
    np.random.seed(0) #make sure split are the same
    labels = np.array(labels)
    labeled_idx = []
    for i in range(classes):
        idx = np.where(labels == i)[0]
        idx = np.random.choice(idx, labeled_per_class, False)
        labeled_idx.extend(idx)
    labeled_idx = np.array(labeled_idx)
    np.random.shuffle(labeled_idx)
    np.random.seed() #optionally restore rng
    return labeled_idx, np.setdiff1d(range(train_total), labeled_idx)

class CIFAR100SSL(datasets.CIFAR100):
    def __init__(self, root, indexs, train=True,
                 transform=None, target_transform=None,
                 download=False):
        super().__init__(root, train=train,
                         transform=transform,
                         target_transform=target_transform,
                         download=download)
        if indexs is not None:
            self.data = self.data[indexs]
            self.targets = np.array(self.targets)[indexs]

    def __getitem__(self, index):
        img, target = self.data[index], self.targets[index]
        img = Image.fromarray(img)

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target


Change to your transfrom

In [ ]:
mean,std = (0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)

transform_labeled = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean,std)])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean,std)])

In [ ]:
root = './tmp'
#adjust this for labeled and unlabed split
labeled_per_class = 100

base_dataset = datasets.CIFAR100(
    root, train=True, download=True)

train_labeled_idxs, train_unlabeled_idxs = x_u_split(base_dataset.targets, labeled_per_class)

train_labeled_dataset = CIFAR100SSL(
    root, train_labeled_idxs, train=True,
    transform=transform_labeled)

train_unlabeled_dataset = CIFAR100SSL(
    root, train_unlabeled_idxs, train=True,
    transform=transform_labeled)

test_dataset = datasets.CIFAR100(
    root, train=False, transform=transform_test, download=False)

Files already downloaded and verified


Change to your batch size

In [ ]:
batch_size = 128
l_train_dataloader = DataLoader(train_labeled_dataset, batch_size=batch_size, shuffle=True)
u_train_dataloader = DataLoader(train_unlabeled_dataset, batch_size=batch_size*5, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=False)

In [ ]:
_, target = next(iter(u_train_dataloader))
print(target.size())

torch.Size([40000])


In [ ]:
print(torch.bincount(target))

tensor([400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400,
        400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400,
        400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400,
        400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400,
        400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400,
        400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400,
        400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400,
        400, 400])


In [ ]:
print(len(train_labeled_dataset))
print(len(train_unlabeled_dataset))
print(len(test_dataset))

10000
40000
10000


In [ ]:
print(any(np.isin(train_labeled_idxs,train_unlabeled_idxs)))

False


In [ ]:
all_idx = np.union1d(train_labeled_idxs,train_unlabeled_idxs)
print(np.setdiff1d(all_idx, range(train_total)))

[]
